# Image Scraping with Stock Photos

This notebook will cover techniques used to scrape images from the websites like Google Images, Unsplash, Pexels, Shutterstock, and Pixabay; using Google's Selenium webdriver in python with Pillow and Requests libraries.

# Importing Libraries and Settting Up the web Driver

In order to use selenium, you must install selenium library and also install the chrome web driver. We will define the path and and initiallize the selenium web driver to begin.

In [25]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
DRIVER_PATH = '/../../../../../../chromedriver'

service = Service(DRIVER_PATH)
service.start()
wd = webdriver.Remote(service.service_url)
wd.quit()

# Google Images

Here we will create a function that will fetch image urls from a specificied web page and return the image URLS in a list.

In [3]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
#     def load_more(wd):
#         wd.fin

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))
    time.sleep(sleep_between_interactions)  
    
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
#             load_more_button = wd.find_element_by_xpath("//input[@value='Show more results'")
#             if load_more_button:
#                 load_more_button.click()
#                 time.sleep(sleep_between_interactions)

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

Now we will create another function to persist the image. This function will take each image URL and convert the image to bytes, then to a RGB file, then saved using haslib'.

In [132]:
def persist_image(folder_path:str,url:str):
    try:
        headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
        image_content = requests.get(url, headers=headers).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

To bring these two functions together, we will need another function that will define the folder where to save the images, how many images to save. Below we create the fuction that will now search and download the images we need.

In [5]:
def search_and_download(search_term:str,driver_path:str,target_path='./images2',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

Now it's time to import our request library, and also the os, io, PIL and hashlib libraries we utilized above. For our purposes, I created a list of search terms that we will iterate through and run our funtion.

In [ ]:
import requests
import os
import io
from PIL import Image
import hashlib

search_terms = ['beard cartoon black and white','beard cartoon color',
                'face outline','face outline color', 'headshot outline']

for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

# Unsplash Free Stock Photo Scrapping

Since that was successful, we can use the same concept and tweak the functions for any other sites. Here we will scrape Unsplash.

In [144]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
DRIVER_PATH = '/Users/jacobtadesse/Documents/LearnCo/computerVision/scrapping/chromedriver 2'

service = Service(DRIVER_PATH)
service.start()
wd = webdriver.Remote(service.service_url)
wd.quit()


You will notice that the search URL is different, and is changed to include the query that we pass in. You can also see that the css selectors we used for google are now different as well.

In [150]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd, scroll_point):  
        wd.execute_script(f"window.scrollTo(0, {scroll_point});")
        time.sleep(sleep_between_interactions)    
 
        
    # build the google query
    search_url = f"https://unsplash.com/s/photos/{query}"

    # load the page
    wd.get(search_url)
    time.sleep(sleep_between_interactions)  
    
    image_urls = set()
    image_count = 0
    number_results = 0
    
    for i in range(1,20):
        scroll_to_end(wd, i*1000)
        time.sleep(5)
        thumb = wd.find_elements_by_css_selector("img._2zEKz")
        time.sleep(5)
        for img in thumb:
            print(img)
            print(img.get_attribute('src'))
            image_urls.add(img.get_attribute('src'))
            image_count = len(image_urls)
            number_results = image_count
            time.sleep(.5)
        print(f"Found: {number_results} search results. Extracting links...")

    return image_urls


We will keep the same formula to persist our files.

In [151]:
def persist_image(folder_path:str,url:str):
    try:
        headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
        image_content = requests.get(url, headers=headers).content
        
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

Now it's time to define a new folder to store these files, unless we want them all in the same repository.

In [152]:
def search_and_download(search_term:str,driver_path:str,target_path='./images-UNSPLASH',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=3)
        
    for elem in res:
        persist_image(target_folder,elem)

Let's give it a go!

In [153]:
import requests
import os
import io
from PIL import Image
import hashlib

search_terms = ['beard', 'male face', 'teen face', 'shaved']

for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="e1ab7e09-43fb-486d-87aa-db1352af2523")>
https://images.unsplash.com/photo-1560580200-32d677b35fba?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="ab4f435b-0635-4055-a2d8-5185f3849629")>
https://images.unsplash.com/photo-1522230130022-498e355165c5?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="f50df83f-d802-433c-bc08-56cc247c914a")>
https://images.unsplash.com/photo-1539063955936-cf9760e4ed30?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="7984b654-1e1e-4900-9e90-e83116d7a765")>
https://images.unsplash.com/photo-1524653736724-8490ee06859d?ixlib=rb-1.2.1&ixid=eyJhcHBf

<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="0af763d4-c0aa-4068-8cfb-5c286c3cb973")>
https://images.unsplash.com/photo-1552045058-11f1c8a4da15?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="35b30443-5ac5-4a7e-826c-7f7f5d327dc4")>
https://images.unsplash.com/photo-1554387415-b0c2fcce17fd?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="8bc48c55-678f-44cb-bf77-11ffa2aad9ec")>
https://images.unsplash.com/photo-1510176263513-f31c60b0528b?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="6609e5e6-e326-401b-a284-31c190e94f04")>
https://images.unsplash.com/photo-1553240799-36bbf332a5c3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOj

<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="785a12a0-a115-4cd1-a8dc-425f3c936d8a")>
https://images.unsplash.com/photo-1555617171-a072c97e09a7?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="f1bd02a5-e694-4b47-a379-aa4815a0b6ef")>
https://images.unsplash.com/photo-1525019461548-85e61dd8e83a?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="dcb60247-7883-45c3-ab41-143d1f2ffe31")>
https://images.unsplash.com/photo-1555952517-2e8e729e0b44?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="5b1cb728-3099-44f1-928c-1e20d3dc763a")>
https://images.unsplash.com/photo-1548946526-f69e2424cf45?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOj

<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="80a4e3c2-67c4-4704-beef-558ecc72e822")>
https://images.unsplash.com/photo-1519058082700-08a0b56da9b4?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="68318b77-59a8-44b6-91f3-db44ae217989")>
https://images.unsplash.com/photo-1480455624313-e29b44bbfde1?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="ce3129e7-b445-45d3-96e1-2c86d5513a36")>
https://images.unsplash.com/photo-1521768291053-f122b7fc3e00?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="3c2442eb-d3c7-4359-aaeb-f33a6f496299")>
https://images.unsplash.com/photo-1569466896818-335b1bedfcce?ixlib=rb-1.2.1&ixid=eyJhc

<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="c582f185-653e-4950-846c-a196b0f8d1b2")>
https://images.unsplash.com/photo-1575767931074-a91868c89acb?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="902808d9-205c-42fa-8665-162a9a57822c")>
https://images.unsplash.com/photo-1515531529054-0d9c31d81de3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="14740288-fbeb-41d3-b961-defc2d78960b")>
https://images.unsplash.com/photo-1564842822972-d50c7a9cfdae?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="52cab827383b48c84fbd3c66a8972b61", element="5cbb3fc6-4c6a-40b9-ab07-8caebb84ebb3")>
https://images.unsplash.com/photo-1503681965625-d2a39869ba05?ixlib=rb-1.2.1&ixid=eyJhc

...

<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="d739599d-7c96-4d92-9183-3b2ecf85ca7b")>
https://images.unsplash.com/photo-1583452209081-100af5fd27ac?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="e01bc9ef-feaa-44d4-9798-cf1dba9adef6")>
https://images.unsplash.com/photo-1529084569259-e885d346f3d9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="a2ecb258-0ba4-48bf-80ff-12f178761f2b")>
https://images.unsplash.com/photo-1563384527750-b79f2337049e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="b643efe4-c53a-40c9-9cf6-2717277959b2")>
https://images.unsplash.com/photo-1571116534010-5f4573037086?ixlib=rb-1.2.1&ixid=eyJhc

<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="d739599d-7c96-4d92-9183-3b2ecf85ca7b")>
https://images.unsplash.com/photo-1583452209081-100af5fd27ac?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="e01bc9ef-feaa-44d4-9798-cf1dba9adef6")>
https://images.unsplash.com/photo-1529084569259-e885d346f3d9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="a2ecb258-0ba4-48bf-80ff-12f178761f2b")>
https://images.unsplash.com/photo-1563384527750-b79f2337049e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="b643efe4-c53a-40c9-9cf6-2717277959b2")>
https://images.unsplash.com/photo-1571116534010-5f4573037086?ixlib=rb-1.2.1&ixid=eyJhc

<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="d739599d-7c96-4d92-9183-3b2ecf85ca7b")>
https://images.unsplash.com/photo-1583452209081-100af5fd27ac?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="e01bc9ef-feaa-44d4-9798-cf1dba9adef6")>
https://images.unsplash.com/photo-1529084569259-e885d346f3d9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="a2ecb258-0ba4-48bf-80ff-12f178761f2b")>
https://images.unsplash.com/photo-1563384527750-b79f2337049e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="b643efe4-c53a-40c9-9cf6-2717277959b2")>
https://images.unsplash.com/photo-1571116534010-5f4573037086?ixlib=rb-1.2.1&ixid=eyJhc

<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="d739599d-7c96-4d92-9183-3b2ecf85ca7b")>
https://images.unsplash.com/photo-1583452209081-100af5fd27ac?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="e01bc9ef-feaa-44d4-9798-cf1dba9adef6")>
https://images.unsplash.com/photo-1529084569259-e885d346f3d9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="a2ecb258-0ba4-48bf-80ff-12f178761f2b")>
https://images.unsplash.com/photo-1563384527750-b79f2337049e?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80
<selenium.webdriver.remote.webelement.WebElement (session="cda61564f8203d71a471c482aeb0294d", element="b643efe4-c53a-40c9-9cf6-2717277959b2")>
https://images.unsplash.com/photo-1571116534010-5f4573037086?ixlib=rb-1.2.1&ixid=eyJhc

SUCCESS - saved https://images.unsplash.com/4/feet.JPG?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNSPLASH/shaved/68cd448447.jpg
SUCCESS - saved https://images.unsplash.com/photo-1480872672282-92927e03e692?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNSPLASH/shaved/a4dfd1b9f2.jpg
SUCCESS - saved https://images.unsplash.com/photo-1529084569259-e885d346f3d9?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNSPLASH/shaved/6ee7d0494e.jpg
SUCCESS - saved https://images.unsplash.com/photo-1579983926774-399a9cb765c3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNSPLASH/shaved/6b58dc59af.jpg
SUCCESS - saved https://images.unsplash.com/photo-1518799175676-a0fed7996acb?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNSPLASH/shaved/639e4683d7.jpg
SUCCESS - saved https://images.unsplash.com/photo-1585581842747-88870a42f144?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&w=1000&q=80 - as ./images-UNS

Success! 

Below we will continue these changes for different sites!

# Pexels.com - Image Scrapping

In [108]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd, scroll_point):  
        wd.execute_script(f"window.scrollTo(0, {scroll_point});")
        time.sleep(sleep_between_interactions)    
 
        
    # build the google query
    search_url = f"https://www.pexels.com/search/{query}/"

    # load the page
    wd.get(search_url)
    time.sleep(sleep_between_interactions)  
    
    image_urls = set()
    image_count = 0
    number_results = 0
    
    for i in range(1,20):
        scroll_to_end(wd, i*10000)
        time.sleep(5)
        thumb = wd.find_elements_by_css_selector("img.photo-item__img")
        for img in thumb:
            print(img)
            print(img.get_attribute('src'))
            image_urls.add(img.get_attribute('src'))
            image_count = len(image_urls)
            number_results = image_count
            time.sleep(5)
            
        print(f"Found: {number_results} search results. Extracting links...")

    return image_urls

In [13]:
def persist_image(folder_path:str,url:str):
    try:
        headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
        image_content = requests.get(url, headers=headers).content
  
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [14]:
def search_and_download(search_term:str,driver_path:str,target_path='./images-Pexels',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [112]:
import requests
import os
import io
from PIL import Image
import hashlib

search_terms = ['beard', 'teenager', 'teen male face']

for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

https://images.pexels.com/photos/2269872/pexels-photo-2269872.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2224699/pexels-photo-2224699.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2714273/pexels-photo-2714273.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/917597/pexels-photo-917597.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/842293/pexels-photo-842293.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/761115/pexels-photo-761115.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2089945/pexels-photo-2089945.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2523812/pexels-photo-2523812.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/532739/pexels-photo-532739.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2080734/pexels-photo-2080734.j

https://images.pexels.com/photos/845434/pexels-photo-845434.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2881253/pexels-photo-2881253.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2183130/pexels-photo-2183130.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2239899/pexels-photo-2239899.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/935987/pexels-photo-935987.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/192473/pexels-photo-192473.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/1187765/pexels-photo-1187765.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2080702/pexels-photo-2080702.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/4332206/pexels-photo-4332206.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/874158/pexels-photo-874158.j

https://images.pexels.com/photos/3965246/pexels-photo-3965246.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/3227322/pexels-photo-3227322.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2743754/pexels-photo-2743754.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2179397/pexels-photo-2179397.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/3215733/pexels-photo-3215733.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/1205405/pexels-photo-1205405.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/995300/pexels-photo-995300.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/69212/pexels-photo-69212.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2745944/pexels-photo-2745944.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/373899/pexels-photo-373899.j

https://images.pexels.com/photos/2269872/pexels-photo-2269872.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2224699/pexels-photo-2224699.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2714273/pexels-photo-2714273.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/917597/pexels-photo-917597.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/842293/pexels-photo-842293.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/761115/pexels-photo-761115.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2089945/pexels-photo-2089945.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2523812/pexels-photo-2523812.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/532739/pexels-photo-532739.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2080734/pexels-photo-2080734.j

https://images.pexels.com/photos/3692739/pexels-photo-3692739.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2955277/pexels-photo-2955277.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/2027665/pexels-photo-2027665.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/3775545/pexels-photo-3775545.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/4034021/pexels-photo-4034021.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/935948/pexels-photo-935948.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/842980/pexels-photo-842980.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/1011529/pexels-photo-1011529.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/1319461/pexels-photo-1319461.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500
https://images.pexels.com/photos/3031391/pexels-photo-30313

...

SUCCESS - saved https://images.pexels.com/photos/3660036/pexels-photo-3660036.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/7f10f91eef.jpg
SUCCESS - saved https://images.pexels.com/photos/2598176/pexels-photo-2598176.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/efa1f409d7.jpg
SUCCESS - saved https://images.pexels.com/photos/3261007/pexels-photo-3261007.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/5ff9279e97.jpg
SUCCESS - saved https://images.pexels.com/photos/3965232/pexels-photo-3965232.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/abb9047470.jpg
SUCCESS - saved https://images.pexels.com/photos/2970499/pexels-photo-2970499.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/6ef9becd10.jpg
SUCCESS - saved https://images.pexels.com/photos/3831569/pexels-photo-3831569.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexel

SUCCESS - saved https://images.pexels.com/photos/745045/pexels-photo-745045.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/fafd8b1b3f.jpg
SUCCESS - saved https://images.pexels.com/photos/2845475/pexels-photo-2845475.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/408069b1f9.jpg
SUCCESS - saved https://images.pexels.com/photos/3756038/pexels-photo-3756038.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/6fcb403ba4.jpg
SUCCESS - saved https://images.pexels.com/photos/3714901/pexels-photo-3714901.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/21f5bae840.jpg
SUCCESS - saved https://images.pexels.com/photos/774098/pexels-photo-774098.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/62cfafb9f0.jpg
SUCCESS - saved https://images.pexels.com/photos/3778680/pexels-photo-3778680.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/te

SUCCESS - saved https://images.pexels.com/photos/3617872/pexels-photo-3617872.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/fa41703aca.jpg
SUCCESS - saved https://images.pexels.com/photos/1038040/pexels-photo-1038040.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/481b726fd7.jpg
SUCCESS - saved https://images.pexels.com/photos/3990842/pexels-photo-3990842.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/23f1cb9bfc.jpg
SUCCESS - saved https://images.pexels.com/photos/1126557/pexels-photo-1126557.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/84a9f05d7d.jpg
SUCCESS - saved https://images.pexels.com/photos/997512/pexels-photo-997512.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/c2fa208c73.jpg
SUCCESS - saved https://images.pexels.com/photos/2383750/pexels-photo-2383750.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/

SUCCESS - saved https://images.pexels.com/photos/3073894/pexels-photo-3073894.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/411c652ab5.jpg
SUCCESS - saved https://images.pexels.com/photos/1435456/pexels-photo-1435456.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/4a660c2893.jpg
SUCCESS - saved https://images.pexels.com/photos/1484794/pexels-photo-1484794.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/1e5f2af55f.jpg
SUCCESS - saved https://images.pexels.com/photos/2838159/pexels-photo-2838159.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/cee775e643.jpg
SUCCESS - saved https://images.pexels.com/photos/702139/pexels-photo-702139.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/acfa59962a.jpg
SUCCESS - saved https://images.pexels.com/photos/3993239/pexels-photo-3993239.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/

SUCCESS - saved https://images.pexels.com/photos/980258/pexels-photo-980258.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/0b0c5c295a.jpg
SUCCESS - saved https://images.pexels.com/photos/3957981/pexels-photo-3957981.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/8309d839d2.jpg
SUCCESS - saved https://images.pexels.com/photos/735551/pexels-photo-735551.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/fc91b684a2.jpg
SUCCESS - saved https://images.pexels.com/photos/2306809/pexels-photo-2306809.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/00334e58dc.jpg
SUCCESS - saved https://images.pexels.com/photos/127229/pexels-photo-127229.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen_male_face/f0cf2456cc.jpg
SUCCESS - saved https://images.pexels.com/photos/3080285/pexels-photo-3080285.jpeg?auto=compress&cs=tinysrgb&dpr=1&w=500 - as ./images-Pexels/teen

# Shutterstock.com - Image Scrapping

In [228]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd, scroll_point = 'document.body.scrollHeight'):  
        wd.execute_script(f"window.scrollTo(0, {scroll_point});")
        time.sleep(sleep_between_interactions)    
    query = query
    max_links_to_fetch = max_links_to_fetch
    wd = wd
    sleep_between_interactions = sleep_between_interactions
        
    # build the google query
    search_url = f"https://www.shutterstock.com/search/{query}"

    # load the page
    wd.get(search_url)
    time.sleep(sleep_between_interactions)  
    
    image_urls = set()
    image_count = 0
    number_results = 0
    
    for i in range(1,10):
        scroll_to_end(wd, 100)
        time.sleep(3)
        thumb = wd.find_elements_by_css_selector("img.z_h_c.z_h_e")
        for img in thumb:
            print(img)
            print(img.get_attribute('src'))
            image_urls.add(img.get_attribute('src'))
            image_count = len(image_urls)
            number_results = image_count
            time.sleep(.5)
            
        print(f"Found: {number_results} search results. Extracting links...")
        
        load_more_button = wd.find_element_by_xpath('//a[@data-automation="BottomNav_NextButton"]')
        print(load_more_button)
        if load_more_button:
            load_more_button.click()
            time.sleep(sleep_between_interactions)        


        else: 
            break
    return image_urls


In [229]:
def persist_image(folder_path:str,url:str):
    try:
        headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
        image_content = requests.get(url, headers=headers).content
  
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [230]:
def search_and_download(search_term:str,driver_path:str,target_path='./images-ShutterStock',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [231]:
import requests
import os
import io
from PIL import Image
import hashlib

search_terms = ['beard', 'teenager', 'teen male face']

for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="c367b77d-9a07-46fa-bab0-886324ad0d79")>
https://image.shutterstock.com/image-photo/perfect-beard-closeup-young-bearded-260nw-1072946747.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="eb958169-1e56-4e80-a6c1-d0f97f948ff2")>
https://image.shutterstock.com/image-vector/set-hairbrush-straight-razor-mustache-260nw-1671659773.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="3d2a1867-099e-4c80-b8fb-e0d2ebe77e4f")>
https://image.shutterstock.com/image-vector/set-vector-bearded-men-faces-260nw-634093157.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="dbf343ab-1aa0-4e94-9f43-8367467161bf")>
https://image.shutterstock.com/image-vector/stylish-beard-moustache-set-collection-260nw-529732243.jpg
<selenium.webdriver.remote.we

<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="507da990-7d72-4353-8e0b-00e7822a4568")>
https://image.shutterstock.com/image-vector/set-different-hipster-haircuts-beards-260nw-376695946.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="748b27b0-4d8e-4f3e-a4f4-03f7ff3860dc")>
https://image.shutterstock.com/image-photo/perfect-style-side-view-young-260nw-311399738.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="5ae2b8a5-2622-4c40-9d1b-48cbcea83e01")>
https://image.shutterstock.com/image-photo/bearded-handsome-man-female-slim-260nw-513442618.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="20554210-4062-4a4b-93e5-ca5af083ccbc")>
https://image.shutterstock.com/image-photo/wide-horizontal-portrait-serious-young-260nw-1542287672.jpg
<selenium.webdriver.remote.webelem

<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="d5a36edd-a6c4-41cc-a271-a3c3fe492f18")>
https://image.shutterstock.com/image-vector/mustache-beard-set-on-white-260nw-368638535.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="581ebfc1-3805-45ef-810d-6cdaf0327946")>
https://image.shutterstock.com/image-photo/smiling-bearded-man-decorated-beard-260nw-1657106104.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="5c33205a-a902-4979-a1cd-27aba61f5d73")>
https://image.shutterstock.com/image-photo/collage-beautiful-portrait-man-full-260nw-631398647.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="cb156ac4-40de-426b-8784-bfaf7dff65d9")>
https://image.shutterstock.com/image-photo/bushy-beard-great-happy-caucasian-260nw-1606409350.jpg
<selenium.webdriver.remote.webelement.W

<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="2cc3f83e-78e7-4a12-8430-452cbd99d92e")>
https://image.shutterstock.com/image-photo/attractive-young-guy-striped-tshirt-260nw-1687848712.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="715ac6e6-e00e-4db1-b661-8deeb12babd0")>
https://image.shutterstock.com/image-photo/man-uniform-cut-beard-hair-260nw-1667248549.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="1b5fcceb-18d4-48bd-ae2a-fcce643af613")>
https://image.shutterstock.com/image-photo/serious-guy-shaving-his-beard-260nw-520565569.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="680c53e8-5db6-409a-a41e-61c5757d46a8")>
https://image.shutterstock.com/image-photo/bearded-man-barber-handsome-long-260nw-1687536583.jpg
<selenium.webdriver.remote.webelement.WebElemen

<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="57102b3f-0e43-492a-b056-4d65fe948b1d")>
https://image.shutterstock.com/image-photo/wondering-what-hipster-appearance-man-260nw-1641269608.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="d19dd131-33dd-495f-ba56-6f99214cb2c0")>
https://image.shutterstock.com/image-photo/strict-face-beard-fashion-barber-260nw-1641268735.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="7125f40f-3a64-4380-98b5-064262895818")>
https://image.shutterstock.com/image-vector/santas-beard-eps8-rgb-global-260nw-764320579.jpg
<selenium.webdriver.remote.webelement.WebElement (session="bc3c23e9277befa6b326dd3695d891cd", element="fded39cf-d960-4941-8776-fcc70b4e3a7f")>
https://image.shutterstock.com/image-photo/man-handsome-hipster-stylish-beard-260nw-1552483574.jpg
<selenium.webdriver.remote.webelement

...

<selenium.webdriver.remote.webelement.WebElement (session="eac24dbee441de18ec6bad97b3bf45bd", element="51ccfe71-4206-4a11-8504-e48988789522")>
https://image.shutterstock.com/image-photo/portrait-student-group-outside-college-260nw-216281437.jpg
<selenium.webdriver.remote.webelement.WebElement (session="eac24dbee441de18ec6bad97b3bf45bd", element="51f9710a-39fa-4511-9b3f-4d2385c708cb")>
https://image.shutterstock.com/image-photo/covering-face-hands-caucasian-boy-260nw-1603965196.jpg
<selenium.webdriver.remote.webelement.WebElement (session="eac24dbee441de18ec6bad97b3bf45bd", element="cc960480-34aa-4e3f-b34d-d577d3feaf6d")>
https://image.shutterstock.com/image-photo/expressive-sensual-mediterranean-italian-dark-260nw-1663903033.jpg
<selenium.webdriver.remote.webelement.WebElement (session="eac24dbee441de18ec6bad97b3bf45bd", element="dc45e8dc-941c-4bc7-802f-c60213a6b5a5")>
https://image.shutterstock.com/image-photo/black-whiteexpressive-mediterranean-italian-dark-260nw-1664667586.jpg
<sele

SUCCESS - saved https://image.shutterstock.com/image-photo/closeup-teen-boy-bathroom-brushing-260nw-317000555.jpg - as ./images-ShutterStock/teen_male_face/d458efc066.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/depressed-teenager-arms-around-his-260nw-228880627.jpg - as ./images-ShutterStock/teen_male_face/53900dc372.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/young-handsome-problematic-boy-looking-260nw-58583605.jpg - as ./images-ShutterStock/teen_male_face/abf6d979c3.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/three-young-friends-happy-isolated-260nw-103678058.jpg - as ./images-ShutterStock/teen_male_face/b86402b519.jpg
SUCCESS - saved https://image.shutterstock.com/image-vector/continuous-line-drawing-set-faces-260nw-1726907911.jpg - as ./images-ShutterStock/teen_male_face/72d91d02e4.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-young-handsome-indian-teenage-260nw-1077212147.jpg - as ./images-ShutterS

SUCCESS - saved https://image.shutterstock.com/image-photo/happy-handsome-young-teen-boy-260nw-1488168287.jpg - as ./images-ShutterStock/teen_male_face/98fc78ec7a.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-confident-young-boy-on-260nw-93382546.jpg - as ./images-ShutterStock/teen_male_face/d88587629d.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/black-white-expressive-mediterranean-italian-260nw-1663902676.jpg - as ./images-ShutterStock/teen_male_face/b2b00728d3.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/asian-teen-boy-black-hair-260nw-1148225144.jpg - as ./images-ShutterStock/teen_male_face/ac57baeff2.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/casual-young-man-portrait-isolated-260nw-25359733.jpg - as ./images-ShutterStock/teen_male_face/fd10e358bc.jpg
SUCCESS - saved https://image.shutterstock.com/image-vector/continuous-line-drawing-set-faces-260nw-1194723013.jpg - as ./images-ShutterStock/teen_male

SUCCESS - saved https://image.shutterstock.com/image-photo/casual-young-man-portrait-isolated-260nw-135471377.jpg - as ./images-ShutterStock/teen_male_face/118a46aa4f.jpg
ERROR - Could not save https://image.shutterstock.com/image-photo/handsome-young-fashion-model-man-260nw-266682860.jpg - cannot identify image file <_io.BytesIO object at 0x10adaa410>
SUCCESS - saved https://image.shutterstock.com/image-vector/vector-illustration-kid-trekking-260nw-1042136530.jpg - as ./images-ShutterStock/teen_male_face/afe7508193.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/black-whiteexpressive-mediterranean-italian-dark-260nw-1664667592.jpg - as ./images-ShutterStock/teen_male_face/02c8f1cb0f.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-handsome-teenage-boy-hoodie-260nw-1663803025.jpg - as ./images-ShutterStock/teen_male_face/eb61a1263b.jpg
SUCCESS - saved https://image.shutterstock.com/image-vector/continuous-line-drawing-set-faces-260nw-1411228499.jpg

SUCCESS - saved https://image.shutterstock.com/image-photo/asian-teenager-thinking-isolated-on-260nw-105230105.jpg - as ./images-ShutterStock/teen_male_face/602f386216.jpg
SUCCESS - saved https://image.shutterstock.com/image-vector/teen-boy-character-portrait-face-260nw-1703276269.jpg - as ./images-ShutterStock/teen_male_face/4c051ed2cb.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/group-young-sporty-people-practicing-260nw-796025338.jpg - as ./images-ShutterStock/teen_male_face/33727a7543.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-player-high-school-soccer-260nw-198943046.jpg - as ./images-ShutterStock/teen_male_face/af7d73a923.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/portrait-smiling-teen-boy-hands-260nw-1681717099.jpg - as ./images-ShutterStock/teen_male_face/cbbefe2a8b.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/asian-man-full-body-shot-260nw-118856860.jpg - as ./images-ShutterStock/teen_male_fac

# Pixabay.com - Image Scrapping

In [241]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd, scroll_point = 'document.body.scrollHeight'):  
        for i in range(1,20):
            wd.execute_script(f"window.scrollTo(0, {i*scroll_point});")
            time.sleep(sleep_between_interactions)    
    query = query
    max_links_to_fetch = max_links_to_fetch
    wd = wd
    sleep_between_interactions = sleep_between_interactions
        
    # build the google query
    search_url = f"https://pixabay.com/images/search/{query}/"

    # load the page
    wd.get(search_url)
    time.sleep(sleep_between_interactions)  
    
    image_urls = set()
    image_count = 0
    number_results = 0
    
    for i in range(1,10):
        scroll_to_end(wd, 1000)
        time.sleep(3)
        thumb = wd.find_elements_by_css_selector("img")
        for img in thumb:
            print(img)
            print(img.get_attribute('src'))
            image_urls.add(img.get_attribute('src'))
            image_count = len(image_urls)
            number_results = image_count
            time.sleep(.5)
            
        print(f"Found: {number_results} search results. Extracting links...")
        
        try:
            load_more_button = wd.find_element_by_xpath('//a[@class="pure-button"]')
            print(load_more_button)
            if load_more_button:
                load_more_button.click()
                time.sleep(sleep_between_interactions)    
            else:
                break
        except: 
            break
    return image_urls


In [242]:
def persist_image(folder_path:str,url:str):
    try:
        headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36'}
        image_content = requests.get(url, headers=headers).content
  
    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [243]:
def search_and_download(search_term:str,driver_path:str,target_path='./images-Pixababy',number_images=200):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [244]:
import requests
import os
import io
from PIL import Image
import hashlib

search_terms = ['beard', 'teenager', 'teen male face']

for search_term in search_terms:
    search_and_download(search_term=search_term, driver_path=DRIVER_PATH)

<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="854defd7-79ec-4b29-8b3d-87235fb57676")>
https://cdn.pixabay.com/photo/2017/12/31/15/56/portrait-3052641__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="d6245fc5-f8e6-434d-8cd7-b99c0fd08e0a")>
https://cdn.pixabay.com/photo/2015/11/16/14/43/cat-1045782__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="a9f23327-c1a7-4368-b2c5-b18982ccc206")>
https://cdn.pixabay.com/photo/2016/11/21/12/42/beard-1845166__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="86a06f48-cf63-4d97-9085-9b3f4f034574")>
https://cdn.pixabay.com/photo/2015/01/27/09/58/human-613601__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="0642215f-cca4-48d6-9ab5-7c7d62e75939")>
https

<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="a78e4c53-a9c4-4713-81c1-c982deb7e178")>
https://cdn.pixabay.com/photo/2016/02/19/10/14/man-1209137__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="16849cf3-22bd-4d74-b792-dfcfa48c8e4e")>
https://cdn.pixabay.com/photo/2016/10/29/20/25/beard-1781443__340.png
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="e35b6376-bb65-4b72-81ee-e16e5d63392f")>
https://cdn.pixabay.com/photo/2012/11/28/01/29/man-67467__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="2c58162b-67e8-4f65-8dff-1193ccc0ac3e")>
https://cdn.pixabay.com/photo/2018/03/26/18/20/man-3263509__340.png
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="71882651-9ff4-4704-9525-e35bb6173af1")>
https://cdn.p

<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="6d1b888f-33dc-4aca-92fc-39b17ec59898")>
https://cdn.pixabay.com/photo/2015/10/12/15/01/man-984093__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="01eed5cd-de0d-4970-86da-de3317015e49")>
https://cdn.pixabay.com/photo/2017/05/03/20/03/mma-2282013__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="74c17d04-3e0e-40c1-8532-1e5378c941de")>
https://cdn.pixabay.com/photo/2016/03/27/07/20/black-and-white-1282264__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="8e10a2af-434a-46ef-86fa-618d4a8e5ae6")>
https://cdn.pixabay.com/photo/2016/11/18/13/19/beard-1834429__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="4df446ae-601f-470b-9143-122fcadde243")>


<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="cfaed63f-10d5-4a23-8f3c-dbb7d71389b9")>
https://cdn.pixabay.com/photo/2014/04/05/11/38/seal-316452__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="b5a52893-0ba1-4107-8b33-47a22d41791d")>
https://cdn.pixabay.com/photo/2017/10/27/23/36/beard-2895837__340.png
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="442bce60-9e69-47ed-88a0-7e8acbf27eb6")>
https://cdn.pixabay.com/photo/2018/01/23/22/19/old-3102577__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="27603df4-d09a-4e0f-a019-10d9e559e21f")>
https://cdn.pixabay.com/photo/2016/11/29/03/07/beard-1866984__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="15f4f7a3-7c83-4b3b-9c1b-4c836da8db8c")>
https://c

<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="8870d77c-e1cb-432a-85f4-e14cac11f7eb")>
https://cdn.pixabay.com/photo/2019/08/08/22/11/gnomes-4393901__340.png
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="6452c062-fd94-4ccc-9069-e207c451805f")>
https://cdn.pixabay.com/photo/2019/08/08/22/11/gnomes-4393908__340.png
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="064c81ea-3cc0-4d89-86f2-87e67097baab")>
https://cdn.pixabay.com/photo/2015/05/11/15/40/prophet-762636__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="ae58aaac-ec72-4618-92bf-6fd57ff4d287")>
https://cdn.pixabay.com/photo/2018/12/10/10/20/animal-3866599__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="5c69d78fc15676c0bfe23c0e480f9ef5", element="09b6e091-4dea-47b5-b5cc-71c2ae75bbac")>
h

...

SUCCESS - saved https://cdn.pixabay.com/photo/2015/01/01/21/40/model-586160__340.jpg - as ./images-Pixababy/teenager/d734b5fde8.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2016/10/12/10/15/girl-1733999__340.jpg - as ./images-Pixababy/teenager/4ceb3a8061.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2012/02/23/08/43/girl-15754__340.jpg - as ./images-Pixababy/teenager/b808881f9f.jpg
ERROR - Could not save https://pixabay.com/static/img/blank.gif - cannot identify image file <_io.BytesIO object at 0x10a4b2e08>
SUCCESS - saved https://cdn.pixabay.com/photo/2018/10/02/11/13/girl-3718526__340.jpg - as ./images-Pixababy/teenager/c5673ca5a9.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2015/09/26/14/07/teenager-959125__340.jpg - as ./images-Pixababy/teenager/044f3b7e28.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2017/10/22/19/19/shirt-2878942__340.jpg - as ./images-Pixababy/teenager/5889fbe6d9.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2014/09/21/03/13/boy-454633__34

SUCCESS - saved https://cdn.pixabay.com/photo/2014/06/26/06/53/girls-377661__340.jpg - as ./images-Pixababy/teenager/2876690bb5.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2019/03/12/12/47/people-4050698__340.jpg - as ./images-Pixababy/teenager/2a7fc4a8ce.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2016/10/12/02/50/girl-1733335__340.jpg - as ./images-Pixababy/teenager/10d4d822e4.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2016/10/12/02/58/girl-1733356__340.jpg - as ./images-Pixababy/teenager/260cb35546.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2014/04/03/10/53/girl-311628__340.png - as ./images-Pixababy/teenager/8c7969d880.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/group-teenage-students-collaborating-on-260nw-749518390.jpg - as ./images-Pixababy/teenager/0e5f33015e.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2015/10/25/17/55/woman-1006100__340.jpg - as ./images-Pixababy/teenager/96c5151979.jpg
SUCCESS - saved https://cdn.pixabay.com/

<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="12cda077-16e5-4b5e-89d3-4528114ae87c")>
https://cdn.pixabay.com/photo/2017/05/29/22/53/sexy-2355191__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="1d74b37a-7886-41b3-bb75-595c55ca1392")>
https://cdn.pixabay.com/photo/2017/08/28/22/13/boy-2691492__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="2689a3af-15fa-472b-a286-82d5f9a7d807")>
https://cdn.pixabay.com/photo/2017/05/29/22/49/sexy-2355167__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="55a25284-6286-41ac-8694-ccee777a301e")>
https://cdn.pixabay.com/photo/2017/08/28/22/12/boy-2691486__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="e532e62b-47ec-471a-ada4-6c0ba0e47bc6")>
https://cdn

<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="a2572b2e-404b-449b-9fea-052a3219bf11")>
https://cdn.pixabay.com/photo/2015/12/23/10/42/boy-1105361__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="4a1eb09b-2a99-4523-98a1-c5ab4fa17c85")>
https://cdn.pixabay.com/photo/2016/01/07/18/56/tough-1126321__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="f49496dc-59b5-4342-9b87-0825f0f99904")>
https://cdn.pixabay.com/photo/2019/06/10/11/18/young-man-4264008__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="2a0033b3-d1d1-4402-9d86-26083ad1ad77")>
https://cdn.pixabay.com/photo/2017/06/05/20/28/boy-2375172__340.jpg
<selenium.webdriver.remote.webelement.WebElement (session="706696e1ca30cfeb000e271eb8ce0a20", element="1b3feb12-ef0f-4ba0-8edc-a054fa50cea8")>
https

SUCCESS - saved https://cdn.pixabay.com/photo/2017/05/29/22/50/sexy-2355174__340.jpg - as ./images-Pixababy/teen_male_face/10d29528d0.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2014/04/02/10/25/boy-303808__340.png - as ./images-Pixababy/teen_male_face/8e2576cb3a.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2019/08/18/21/00/boy-4415053__340.jpg - as ./images-Pixababy/teen_male_face/7934585ce9.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2017/05/29/22/49/sexy-2355166__340.jpg - as ./images-Pixababy/teen_male_face/6b49d9d628.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2017/05/29/22/50/sexy-2355176__340.jpg - as ./images-Pixababy/teen_male_face/a30eefe4f5.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2018/09/19/07/14/teenager-3687941__340.jpg - as ./images-Pixababy/teen_male_face/1e5c42fc4a.jpg
SUCCESS - saved https://cdn.pixabay.com/photo/2017/08/28/22/13/boy-2691489__340.jpg - as ./images-Pixababy/teen_male_face/612313b847.jpg
SUCCESS - saved https://cdn.pixaba